In [1]:
import os
from scipy import misc
import numpy as np
from matplotlib import pyplot as plt
from keras.utils.data_utils import get_file
from six.moves import cPickle
import sys
from keras.utils import np_utils 
import cPickle as pickle
from sklearn.cross_validation import train_test_split
import random

Using Theano backend.


In [4]:
#define a list of used labels
usedclassif = ['consolidation',
 'fibrosis',
 'ground_glass',
 'healthy',
 'micronodules',
 'reticulation']
print (usedclassif)

['consolidation', 'fibrosis', 'ground_glass', 'healthy', 'micronodules', 'reticulation']


In [5]:
#define a dictionary with labels
classif={}
i=0
for f in usedclassif:
    classif[f]=i
    i+=1

In [6]:
print classif

{'micronodules': 4, 'healthy': 3, 'ground_glass': 2, 'consolidation': 0, 'reticulation': 5, 'fibrosis': 1}


In [7]:
# define the current work directory
cwd=os.getcwd()
cwd

'C:\\Users\\sylvain\\Documents\\boulot\\startup\\radiology\\workarea\\trainingarea'

In [8]:
# create path to patch directory
patch_dir = os.path.join(cwd, 'patches')

In [9]:
# list all directories under patch directory. They are representing the categories
category_list = (os.listdir(patch_dir))
##print 'taking out the item : ', category_list.pop(0)

In [10]:
# print what we have as categories
category_list

['air_trapping',
 'bronchial_wall_thickening',
 'bronchiectasis',
 'consolidation',
 'cysts',
 'early_fibrosis',
 'emphysema',
 'fibrosis',
 'ground_glass',
 'healthy',
 'increased_attenuation',
 'macronodules',
 'micronodules',
 'Nouveau document texte.txt',
 'pcp',
 'peripheral_micronodules',
 'reticulation',
 'tuberculosis']

In [11]:
# creating variables
# list for the merged pixel data
dataset_list = []
# list of the label data
label_list = []



# go through all categories
# 
for category in category_list:
    if category in usedclassif:
        category_dir = os.path.join(patch_dir, category)
        print  'the path into the categories is: ', category_dir

        sub_categories_dir_list = (os.listdir(category_dir))
        print 'the sub categories are : ', sub_categories_dir_list

        for subCategory in sub_categories_dir_list:

            subCategory_dir = os.path.join(category_dir, subCategory)
            print  'the path into the sub categories is: '
            print subCategory_dir

            image_files = (os.listdir(subCategory_dir))

            for file in image_files:

                if file.find('.bmp') > 0:

                    # load the .bmp file into array       
                    image = misc.imread(os.path.join(subCategory_dir,file), flatten= 0)
                    # append the array to the dataset list
                    dataset_list.append(image)

                    # created rotated copies of images
                    image90 = np.rot90(image)
                    dataset_list.append(image90)

                    image180 = np.rot90(image90)
                    dataset_list.append(image180)

                    image270 = np.rot90(image180)
                    dataset_list.append(image270)

                    # append  lables 
                    label_list.append(classif[category])
                    # append also lables for rotated images
                    label_list.append(classif[category])
                    label_list.append(classif[category])
                    label_list.append(classif[category])


            

the path into the categories is:  C:\Users\sylvain\Documents\boulot\startup\radiology\workarea\trainingarea\patches\consolidation
the sub categories are :  ['apical', 'diffuse', 'non-relevant', 'peripheral_sub_pleural']
the path into the sub categories is: 
C:\Users\sylvain\Documents\boulot\startup\radiology\workarea\trainingarea\patches\consolidation\apical
the path into the sub categories is: 
C:\Users\sylvain\Documents\boulot\startup\radiology\workarea\trainingarea\patches\consolidation\diffuse
the path into the sub categories is: 
C:\Users\sylvain\Documents\boulot\startup\radiology\workarea\trainingarea\patches\consolidation\non-relevant
the path into the sub categories is: 
C:\Users\sylvain\Documents\boulot\startup\radiology\workarea\trainingarea\patches\consolidation\peripheral_sub_pleural
the path into the categories is:  C:\Users\sylvain\Documents\boulot\startup\radiology\workarea\trainingarea\patches\fibrosis
the sub categories are :  ['apical', 'basal', 'diffuse', 'non-releva

In [12]:
print (len(dataset_list),len(label_list))



(25572, 25572)


In [13]:
nb_elem = 25000 
indices = []  
resultatx=[]
resultaty= [] 
while nb_elem > 0:  
    i = random.randint(0, len(dataset_list) -1)  
    while i in indices: # tant que le tirage redonne un nombre déjà choisi  
        i = random.randint(0, len(dataset_list) -1)  
    indices.append(i)  
    nb_elem = nb_elem - 1  

for index in indices:  
    resultatx.append(dataset_list[index])  
    resultaty.append(label_list[index]) 

# transform dataset list into numpy array                   
dataset = np.array(resultatx)

# use only one of the 3 color channels as greyscale info
X = dataset[:,:, :,1]

print 'dataset shape is now: ', X.shape

# 
y = np.array(resultaty)
# sampling item 22
#y[22]


dataset shape is now:  (25000L, 32L, 32L)


In [14]:
print (len(dataset_list),len(label_list))
print (len(resultatx),len(resultaty))

(25572, 25572)
(25000, 25000)


In [15]:
print X.shape
print y.shape

(25000L, 32L, 32L)
(25000L,)


In [16]:
X_train, X_intermediate, y_train, y_intermediate = train_test_split(X, y, test_size=0.5, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_intermediate, y_intermediate, test_size=0.5, random_state=42)
print X_train.shape
print X_val.shape
print X_test.shape
print y_train.shape
print y_test.shape
print y_val.shape
    

(12500L, 32L, 32L)
(6250L, 32L, 32L)
(6250L, 32L, 32L)
(12500L,)
(6250L,)
(6250L,)


In [17]:
# save the dataset and label set into serial formatted pkl 

pickle.dump(X_train, open( "X_train.pkl", "wb" ))
pickle.dump(X_test, open( "X_test.pkl", "wb" ))
pickle.dump(X_val, open( "X_val.pkl", "wb" ))
pickle.dump(y_train, open( "y_train.pkl", "wb" ))
pickle.dump(y_test, open( "y_test.pkl", "wb" ))
pickle.dump(y_val, open( "y_val.pkl", "wb" ))

In [18]:
# testing if pickls was working fine
recuperated_X_train = pickle.load( open( "X_train.pkl", "rb" ) )

In [113]:
recuperated_X_train

array([[[ 38,   7,   2, ...,  49,  44,  66],
        [  1,  25,  19, ...,  38,  16,   0],
        [  0,  14,  34, ...,  80,  57,  10],
        ..., 
        [ 47,  22,  48, ...,  26,  32,  23],
        [ 44,   7,  12, ...,  43,  30,   6],
        [ 28,  20,  14, ...,  46,  25,  44]],

       [[  0,   0,   0, ...,   0,   0,   0],
        [  0,   0,   0, ...,   0,   0,   0],
        [  0,   0,   0, ...,   0,   0,   0],
        ..., 
        [  0,   0,   0, ...,   0,   0,   0],
        [  0,   0,   0, ...,   0,   0,   0],
        [  0,   0,   0, ...,   0,   0,   0]],

       [[  0,   0,   0, ...,   0,   0,   0],
        [  0,   0,   0, ...,   0,   0,   0],
        [  0,   0,   0, ...,   0,   0,   0],
        ..., 
        [  0,   0,   0, ...,   0,   0,   0],
        [  0,   0,   0, ...,   0,   0,   0],
        [  0,   0,   0, ...,   0,   0,   0]],

       ..., 
       [[  0,   0,   0, ...,   0,   0,   0],
        [  0,   0,   0, ...,   0,   0,   0],
        [  0,   0,   0, ...,   0,   0, 